In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

def plus(inputs):
  a,b = inputs.split(',')
  return float(a) + float(b)

agent = initialize_agent(
  llm=llm, 
  verbose=True,
  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
  handle_parsing_errors=True,
  tools=[
    Tool.from_function(
      func=plus,
      name="Sum Calculator",
      description="Use this to perfom sums of two numbers. Use this tool by snedgin a pair of number separated by a comma.\n Example:1,2"
  ),
])

prompt = "Cost of $123.42 + $12932.12 + $121.3 + $120.59 + $1940.29 + $59.11"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
I should use the Sum Calculator to add up all the numbers.
Action: Sum Calculator
Action Input: 123.42,12932.12
Observation: 13055.54
Thought:I need to continue adding the remaining numbers.
Action: Sum Calculator
Action Input: 13055.54,121.3
Observation: 13176.84
Thought:I need to keep adding the remaining numbers.
Action: Sum Calculator
Action Input: 13176.84,120.59
Observation: 13297.43
Thought:I need to add the last two numbers.
Action: Sum Calculator
Action Input: 13297.43,1940.29
Observation: 15237.720000000001
Thought:I now know the final answer
Final Answer: $15237.72

> Finished chain.


{'input': 'Cost of $123.42 + $12932.12 + $121.3 + $120.59 + $1940.29 + $59.11',
 'output': '$15237.72'}